In [ ]:
# Outline of Pipeline
# 1. Get CT source points from vessel extraction
# 1.1 Get tomoRecon points from hand drawn extraction
# 2. Project tomorRecon points onto tomosynthesis projection (using center of emitter, 1 position)
# 3. Make mask with drawn tomoRecon points
# 4. Apply the mask to each of the tomosynthesis projection images
# 5. Get masked tomosynthesis images
# 6. Project and register CT source points with the mask applied to the tomosynthesis projection

In [ ]:
import PythonVersorRigid3DPerspectiveTransform as T
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.optimize import least_squares
import time
import itk
from itkwidgets import view
import os
from pathlib import Path
import functions as func

In [ ]:
# GET FILES
# path to relevent directories and files

In [ ]:
# patient 2
tomoProj_dir = "./Data/TomoProjection_02"
tomoRecon_dir = "./Data/TomoReconstruction_02"
overlay_dir = "./Data/TomoAnnotation_02"
emitterGeo_file = "./Data/TomoProjection_02/geo.txt"
vessel_file = "./Data/Vessels_02/CT-Lungs-Vessels_02.tre"
ct_file = "./Data/CT_02/CT_02_04.nii"
MaskDir = "./Results/Masks"
MaskTomoDir = "./Results/MaskedTomo"
SolutionTomoDir = "./Results/SolutionTomo"
ResultsDir = "./Results/"
solution_output_filename = "regSolution_02.txt"
x_init = [0, 55, -25, math.pi, math.pi, 0]
img = itk.imread(ct_file)
spacing = img.GetSpacing()
spacing = spacing[0]

In [ ]:
# number of tomo images and recon files
tomoFileNumber = 36
tomoReconFileNumber = 86

# get list of paths to tomo images (mirrored) and tomoRecon images
tomo_preproccess_list = []
tomo_proj_list = []
mask_list = []
tomo_recon_list = []
overlay_list = []

for i in range(tomoReconFileNumber):
    new_suffix = f'{i+1:03}.dcm'
    file_name = tomoRecon_dir+"/Image_"+new_suffix
    file_path = Path(file_name)
    if os.path.isfile(file_path):
        tomo_recon_list.append(file_name)

for i in range(tomoFileNumber):
    new_suffix = f'{i+1:02}.dcm'
    file_name = tomoProj_dir+"/Image_"+new_suffix
    file_path = Path(file_name)
    if os.path.isfile(file_path):
        tomo_preproccess_list.append(file_name)    

for i in range(tomoFileNumber):
    new_suffix = f'{i+1:03}.dcm'
    file_name = overlay_dir+"/overlayMask_"+new_suffix
    file_path = Path(file_name)
    if os.path.isfile(file_path):
        overlay_list.append(file_name)

# preproccess tomo projections (get mirrored image), and get list of paths 
for path, i in zip(tomo_preproccess_list, range(len(tomo_preproccess_list))):
    img = itk.imread(path)
    img = itk.GetArrayFromImage(img)
    img = np.squeeze(img)
    img = np.fliplr(img)
    img = itk.GetImageFromArray(img)
    file_name = tomoProj_dir+"/FlippedImage_{}.mha".format(f'{i+1:02}')
    file_path = Path(file_name)
    itk.imwrite(img, file_path)
    
for i in range(tomoFileNumber):
    new_suffix = f'{i+1:02}.mha'
    file_name = tomoProj_dir+"/FlippedImage_"+new_suffix
    file_path = Path(file_name)
    if os.path.isfile(file_path):
        tomo_proj_list.append(file_name)  
    
# get list of emitter positions
with open(emitterGeo_file) as f:
    lines = f.readlines()
    geo_lines = []
    for line in lines:
        line = str(line)
        line = line.strip("\n").strip("'")
        line = line.split(" ")
        while("" in line) :
            line.remove("")
        geo_lines.append(line)
# first three are "fit" - disregard first line
del geo_lines[0]
for i in geo_lines:
    del i[3:6]
    for num in range(len(i)):
        i[num] = float(i[num])

In [ ]:
# GLOBAL VARIABLES
# define x_scale for lsq optimization
x_scale = np.array([1/13000000, 1/13000000, 1/13000000, .000001, .000001, .000001])
# size of tomo projection
size = itk.size(itk.imread(tomo_proj_list[3]))
# size of tomo reconstruction
sizeRecon = itk.size(itk.imread(tomo_recon_list[3]))
# define initial x values
x0 = [0, 0, 0, 0, 0, 0]

#################################
# Set up VersorRigid3DPerspectiveTransform for projectng TomosynthesisRecon Data 
CenterOfRotation = np.array([0, 0, 0])                         # not used in the transform point (using center of field)
PlaneCenter = np.array([sizeRecon[0]/2, sizeRecon[1]/2, 0])    # center of detector
PlaneNormal = np.array([0, 0, 1])                              # vector normal
xDirection = np.array([sizeRecon[0]/2+1, sizeRecon[1]/2, 0])   # vector x direction
yDirection = np.array([sizeRecon[0]/2, sizeRecon[1]/2+1, 0])   # vector y direction
line_15 = geo_lines[15]                                        # define projection position for general calcs/visualization
EmitterPosition = [(sizeRecon[0]/2)*0.194-line_15[0], (sizeRecon[1]/2)*0.194+line_15[1], line_15[2]]
#################################
project = T.VersorRigid3DPerspectiveTransform(CenterOfRotation, EmitterPosition, PlaneCenter, PlaneNormal, xDirection, yDirection)

#################################
# Set up VersorRigid3DPerspectiveTransform for projecting CT Data
CenterOfRotation = np.array([0, 0, 0])                     # not used in the transform point (using center of field)
PlaneCenter = np.array([0, 0, 0])                          # center of detector
PlaneNormal = np.array([0, 0, 1])                          # vector normal
xDirection = np.array([1, 0, 0])                           # vector x direction
yDirection = np.array([0, 1, 0])                           # vector y direction
EmitterPosition =[-line_15[0], line_15[1], line_15[2]]
#################################
projectCT = T.VersorRigid3DPerspectiveTransform(CenterOfRotation, EmitterPosition, PlaneCenter, PlaneNormal, xDirection, yDirection)

In [ ]:
# 1. Get CT source points from vessel extraction
Dimension = 3

reader = itk.SpatialObjectReader[Dimension].New()
reader.SetFileName(vessel_file)
reader.Update()
tubes = reader.GetGroup()

castSO = itk.CastSpatialObjectFilter[3].New()
castSO.SetInput(tubes)
tubesSO = castSO.GetTubes()
points = []
radii = []

for i in range(tubes.GetNumberOfChildren()):
    tube = tubesSO[i]
    p = list(map(lambda x: tube.GetPoint(x).GetPositionInObjectSpace(), range(tube.GetNumberOfPoints())))
    for i in p:
        points.append(i)
    r = list(map(lambda x: tube.GetPoint(x).GetRadiusInObjectSpace(), range(tube.GetNumberOfPoints())))
    for j in r:
        radii.append(j)

points = np.array(points)
radii = np.array(radii)
source_points = np.ones([len(points), 3])
for i, point in zip(range(len(points)), points):
    source_points[i] = [point[0], point[2], point[1]]

view(image=itk.imread(ct_file), point_sets=tubes)

In [ ]:
# Perform initial transform on CT Points and get subset of CT source points to use for optimzation
# visualize for verification
source = func.TransformAllPointsAllParameters(x_init, source_points[1::100])
projectedPoints, _, _= func.GetProjectedPointsCTTP(source, projectCT, size, spacing)
plt.close()
plt.rcParams["figure.figsize"] = (12,8.5)
img = itk.imread(tomo_proj_list[15])
img = np.squeeze(img)
fig, ax = plt.subplots()
ax.imshow(img)
for point in projectedPoints:
    line1 = plt.scatter(point[0], point[1], 5, c='y')
plt.show()

In [ ]:
# 1.1 Get tomoRecon points from hand drawn extraction
mask = []
for vessel_annotation_file, i in zip(overlay_list, range(len(overlay_list))):
    img = itk.imread(vessel_annotation_file)
    img.SetSpacing((0.194, 0.194, 1))
    imgarr = itk.GetArrayFromImage(img)
    point = np.argwhere(imgarr>0)
    for index in point:
        mask.append([index[2]*0.194, index[1]*0.194, (35+i)*3])
mask = np.array(mask)

In [ ]:
# 2. Project tomorRecon points onto tomosynthesis projection (using center of emitter, 1 position)
# check to make sure it projects to generally right area
projectedPoints, _, _= func.GetProjectedPointsTRTP(mask, project, sizeRecon)

plt.close()
plt.rcParams["figure.figsize"] = (12,8.5)
img = itk.imread(tomo_proj_list[15])
img = np.squeeze(img)
fig, ax = plt.subplots()
ax.imshow(img)
for point in projectedPoints[1::200]:
    line1 = plt.scatter(point[0], point[1], 5, c='y')
plt.show()

In [ ]:
# 3. Make mask with drawn tomoRecon points
func.MakeVesselMask(geo_lines, mask, size, MaskDir)

In [ ]:
# 4. Apply the mask to each of the tomosynthesis projection images
for i in range(len(tomo_proj_list)):
    inputIm = itk.imread(tomo_proj_list[i], itk.F)
    inputIm = np.squeeze(inputIm)
    inputIm = itk.GetImageFromArray(inputIm)
    maskCT = itk.imread(MaskDir+"/mask_{}.dcm".format(f'{i+1:02}'), itk.UC)
    maskCT = np.squeeze(maskCT)
    maskCT = maskCT.astype(dtype=np.uint8)
    maskCT = itk.GetImageFromArray(maskCT)

    ImageType = itk.Image[itk.F, 2]
    MaskType = itk.Image[itk.UC, 2]
    MaskFilterType = itk.MaskImageFilter[ImageType, MaskType, ImageType]
    maskFilter = MaskFilterType.New()
    maskFilter.SetInput(inputIm)
    maskFilter.SetMaskImage(maskCT)
    maskFilter.Update()
    itk.imwrite(maskFilter.GetOutput(), MaskTomoDir+"/maskedTomo_{}.mha".format(f'{i+1:02}'))

In [ ]:
# 5. Get masked tomosynthesis images
for i in range(36):
    file_name = MaskTomoDir+"/maskedTomo_{}.mha".format(f'{i+1:02}')
    file_path = Path(file_name)
    if os.path.isfile(file_path):
        mask_list.append(file_name)    

In [ ]:
# 6. Project and register CT source points with the mask applied to the tomosynthesis projection
u = np.array(source[1::5])
y = geo_lines

t0 = time.time()
res_trf = least_squares(func.CT_TomoProjectionRegistration, x0*x_scale, args=(u, y, mask_list, x_scale, size, spacing), 
                        verbose=2, method='trf', tr_options={"regularize": False}, 
                        bounds=[-3, 3], diff_step=[0.9, 0.9, 0.9, 0.1, 0.1, 0.1],
                        gtol=1e-15, xtol=1e-15)
t1 = time.time()
print("Optimization took {} seconds".format(t1 - t0))

In [ ]:
# define data
regSolution = np.asarray(res_trf.x/x_scale)
# save to csv file
np.savetxt(ResultsDir+solution_output_filename, regSolution, delimiter=',')

In [ ]:
source = func.TransformAllPointsAllParameters(x_init, source_points[1::100])
projectedPoints, _, _= func.GetProjectedPointsCTTP(source, projectCT, size, spacing)
transformed = func.TransformAllPointsAllParameters(res_trf.x/x_scale, source)
projectedPoints_opt, _, _ = func.GetProjectedPointsCTTP(transformed, projectCT, size, spacing)

plt.close()
plt.rcParams["figure.figsize"] = (12,8.5)
img = itk.imread(mask_list[15])
img = np.squeeze(img)
fig, ax = plt.subplots()
ax.imshow(img)
for point, opt in zip(projectedPoints, projectedPoints_opt):
    line1 = plt.scatter(point[0], point[1], 10, c='y')
    line2 = plt.scatter(opt[0], opt[1], 10, c='r')
ax.legend([line1, line2], ['Initial', 'Opt'])
plt.show()

In [ ]:
# # Make 2D transformed vessel overlay 
source = func.TransformAllPointsAllParameters(x_init, source_points[1::20])
transformed = func.TransformAllPointsAllParameters(res_trf.x/x_scale, source)
func.MakeVesselOverlay(geo_lines, transformed, size, SolutionTomoDir, spacing)